# Adding river levels
*Developed by R.A. Collenteur & D. Brakenhoff*

In this example it is shown how to create a Pastas model that not only includes precipitation and evaporation, but also observed river levels. We will consider observed heads that are strongly influenced  by river level, based on a visual interpretation of the raw data.

In [ ]:
import pandas as pd
import pastas as ps
import matplotlib.pyplot as plt

ps.show_versions()
ps.set_log_level("INFO")

## 1. import and plot data
Before a model is created, it is generally a good idea to try and visually interpret the raw data and think about possible relationship between the time series and hydrological variables. Below the different time series are plotted.

The top plot shows the observed heads, with different observation frequencies and some gaps in the data. Below that the observed river levels, precipitation and evaporation are shown. Especially the river level show a clear relationship with the observed heads. Note however how the range in the river levels is about twice the range in the heads. Based on these observations, we would expect the the final step response of the head to the river level to be around 0.5 [m/m]. 

In [ ]:
oseries = pd.read_csv("data/nb5_head.csv", parse_dates=True, index_col=0).squeeze()
rain = pd.read_csv("data/nb5_prec.csv", parse_dates=True, index_col=0).squeeze()
evap = pd.read_csv("data/nb5_evap.csv", parse_dates=True, index_col=0).squeeze()
waterlevel = pd.read_csv("data/nb5_riv.csv", parse_dates=True, index_col=0).squeeze()

ps.plots.series(oseries, [rain, evap, waterlevel], figsize=(10, 5), hist=False);

## 2. Create a timeseries model
First we create a model with precipitation and evaporation as explanatory time series. The results show that precipitation and evaporation can explain part of the fluctuations in the observed heads, but not all of them.

In [ ]:
ml = ps.Model(oseries.resample("D").mean().dropna(), name="River")

sm = ps.RechargeModel(rain, evap, rfunc=ps.Exponential(), name="recharge")
ml.add_stressmodel(sm)

ml.solve(tmin="2000", tmax="2019-10-29")
ml.plots.results(figsize=(12, 8));

## 3. Adding river water levels
Based on the analysis of the raw data, we expect that the river levels can help to explain the fluctuations in the observed heads. Here, we add a stress model (`ps.StressModel`) to add the rivers level as an explanatory time series to the model. The model fit is greatly improved, showing that the rivers help in explaining the observed fluctuations in the observed heads. It can also be observed how the response of the head to the river levels is a lot faster than the response to precipitation and evaporation.

In [ ]:
w = ps.StressModel(waterlevel, rfunc=ps.One(), name="waterlevel", settings="waterlevel")
ml.add_stressmodel(w)
ml.solve(tmin="2000", tmax="2019-10-29")
axes = ml.plots.results(figsize=(12, 8))
axes[-1].set_xlim(0, 10);  # By default, the axes between responses are shared.